In [1]:
pip install geopandas

     |████████████████████████████████| 1.0 MB 4.3 MB/s 
     |████████████████████████████████| 6.3 MB 60.8 MB/s 
     |████████████████████████████████| 15.4 MB 74.2 MB/s 


In [113]:
#Import necessary libraries
import pandas as pd
import numpy as np
import geopandas as gpd

In [114]:
#Import dataset for townland, County, Electoral
townland_df = pd.read_csv('Townlands.csv') # for Townland
county_df=pd.read_csv('County.csv') # For County
town_df=pd.read_csv('Electoral.csv') # For Town

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [116]:
#Select only relevant columns
town_df=town_df[["English_Name", "ITM_E","ITM_N"]]
county_df=county_df[["English_Name", "ITM_E","ITM_N"]]
townland_df=townland_df[["English_Name", "ITM_E","ITM_N"]]

In [157]:
#Function to find Townland
def find_townland_(townland_name,town_name,county_name):
  townland_filtered_df=townland_df.query('English_Name == @townland_name')
  townland_count=len(townland_filtered_df.index)
  if(townland_count==1):
    #Return Exact match
    return (townland_filtered_df)
  elif(townland_count>1):
    found_town_df=find_town_(town_name,county_name)
    return find_closest_coordinates(townland_filtered_df,found_town_df)
  else:
    #no match found for this townloand , look for the relevant town and return co-ordinates
    return find_town_(town_name,county_name)

In [155]:
#Function to find Town
def find_town_(town_name, county_name) :
  town_filtered_df=town_df.query('English_Name == @town_name')
  town_count=len(town_filtered_df.index)
  if(town_count==1): #Found exact match
    return(town_filtered_df)
  elif(town_count>1):
    found_county_df=find_county(county_name)
    return find_closest_coordinates(town_filtered_df,found_county_df)
  else:
    #no match found for given town, search for relevant county
    return find_county(county_name)

In [300]:
#Function to find county
def find_county(county_name):
  county_filtered_df=county_df.query('English_Name == @county_name')
  #As there is only 1 county, we won't check for duplicates 
  if(len(county_filtered_df.index)<1):
    list_query=[county_name, 0, 0]
    return pd.DataFrame([list_query], columns = ['English_Name','ITM_E','ITM_N'] )
  return (county_filtered_df)

In [222]:
#Function to find the closest co-ordinates for the given location
def find_closest_coordinates(parentdf,childdf):
  ITM_E_df=parentdf.iloc[(parentdf['ITM_E']-childdf.iloc[0]['ITM_E']).abs().argsort()[:1]]
  ITM_N_df=parentdf.iloc[(parentdf['ITM_N']-childdf.iloc[0]['ITM_N']).abs().argsort()[:1]]
  df_merge = pd.merge(ITM_E_df, ITM_E_df, how='inner')
  return(df_merge)

In [307]:
#Function to change format of the input string
def change_location_format_(query):
  list_query = query.upper().split(",")
  if(len(list_query)<3): #If there is no townland
    townland=''
  else:
    townland=list_query[-3].strip()
  town=list_query[-2].strip()
  county=list_query[-1].strip()
  return ""+townland+','+town+','+county+""


In [306]:
#To select only english_name,ITM_E,ITM_N from the received output
def select_value_only_(df):
  return (df.iloc[0]['English_Name'],df.iloc[0]['ITM_E'],df.iloc[0]['ITM_N'])

In [301]:
test_df=pd.read_csv('addresses_for_task.csv') #Test file to check Output

In [302]:
#Convert the given address to uppercase and separate into 3 diff columns using delimeter ,
test_df['Address_Upper'] = test_df.apply(lambda row : change_location_format_(row['Address']), axis = 1)
test_df[['townland', 'town', 'county']] =test_df['Address_Upper'].str.split(',', expand=True)
# Column output will conatin the geo co-ordinates for closest location
test_df['Output'] = test_df.apply(lambda row : (select_value_only_(find_townland_(row['townland'],
                                  row['town'],row['county']))), axis = 1)
#Converting Output into 3 separate columns
new_col_list = ['Clostest_Location', 'ITM_E', 'ITM_N']
for n,col in enumerate(new_col_list):
    test_df[col] = test_df['Output'].apply(lambda location: location[n])

In [303]:
#Function to convert ITM coordinates in to the usual Geo co-ordinates
def create_geodf_from_GPS (df, x, y, crs):

   locations = gpd.points_from_xy(x, y)
   geo_df = gpd.GeoDataFrame(df, geometry=locations)
   geo_df.crs = crs
   return geo_df

In [305]:
test_df_geo = create_geodf_from_GPS(test_df, x=test_df["ITM_E"], y=test_df["ITM_N"], crs = 'epsg:2157')
test_df_geo = test_df_geo.to_crs('epsg:4326')
test_df_geo=test_df_geo[["Address", "ITM_E","ITM_N","geometry"]]
test_df_geo.rename(columns={'geometry':'Geo Co-ordinates'}, inplace=True)
test_df_geo.to_csv('Address_Output.csv',index=False)